<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: skymatch and photometry with MIRI 

<span style="color:red"> **Instruments Affected**</span>: MIRI, NIRCam 


Tested on MIRI Simulated data

### Table of Contents
<div style="text-align: left"> 

<br> [Introduction](#intro_ID) 
<br> [JWST CalWG Algorithm](#algorithm)
<br> [Test Description](#description)
<br> [Data Description](#data_descr)
<br> [Imports](#imports_ID)
<br> [Loading the Data](#data_load)
<br> [Set up association files](#associations_ID)
<br> [Insert outliers](#outliers_ID) 
<br> [Run Pipeline](#pipeline_ID) 


</div>

<a id="intro_ID"></a>
## Introduction

This notebook is meant to test how the differing methods in the skymatch step affect photometry output in a catalog at the end of the source_catalog step. This is set up to test each of the skymatch methods (local, global, match, and global+match) and see if there are differences in photometry between them. The plot at the end of the notebook compare the total AB mag calculated in the source_catalog step by x position. If there are no differences, the magnitudes for each method should line up. If there are differences between them, it will show in different symbols not lining up.

This notebook runs the following steps only from calwebb_image3: skymatch, resample and source_catalog, checking outputs of each step along the way. The final result is the plot at the bottom of the notebook as described above.


The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/skymatch/README.html

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst/tree/master/jwst/skymatch


<a id="algorithm"></a>
# JWST CalWG Algorithm

The skymatch step can be used to compute sky values in a collection of input images that contain both sky and source signal. The sky values can be computed for each image separately or in a way that matches the sky levels amongst the collection of images so as to minimize their differences. This operation is typically applied before combining multiple images into a mosaic. 

Background Matching algorithm page on the CalWG confluence page
> https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Image+Background+Matching


[Top of Page](#title_ID)

<a id="description"></a>
# Test Description

This notebook processes level 2 images through the calwebb_image3 skymatch and resample steps and examines outputs for different sky method parameters.

1) Set up data path and image list file.

2) Set up association files.

3) Modify average backgrould level of input images.

4) Run skymatch step on images.

5) Run skymatch, resample and source_catalog for each skymethod parameter (local, global, match, global+match). 

6) See if the differing skymatch methods affect the source photometry.

7) Testing other parameters (nclip, usigma, lsigma, lower, upper, skystat). (Still TBD.)

This notebook has been created to test how skymatch, resample and source_catalog work together (with subtract=True indicating that the subtraction is done in the skymatch step and subtract=False indicating that the subtraction is done in the resample step). There are displays of the combined noise image to see how well the backgrounds were subtracted in the full combined image.

[Top of Page](#title_ID)

<a id="data_descr"></a>
# Data Description
These steps are set up with simulated MIRI F560W data of a crowded field with a variety of point sources and extended sources. There are four dither positions of the same field that are combined in the resample step. The notebook adds noise backgrounds to the field to help the pipeline testers have a better visualization of what the pipeline is doing with each set of parameters. 

[Top of Page](#title_ID)

### Set up temporary directory and CRDS options

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmp4ujd505z'>


In [2]:
# Set up CRDS options

import os
if "CRDS_CACHE_TYPE" in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']

<a id="imports_ID"></a>
### Set up import statements

The following packages are needed to run this notebook:
* jwst to run the pipeline, access datamodels and create associations 
* matplotlib to plot
* astropy for a variety of calculations
* glob to find files of a certain type
* imports to retrieve files from Box


In [3]:
import pytest
from astropy.io import fits
from astropy.table import Table, vstack, unique, join
from astropy import table
from glob import glob

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

import jwst
import json
from jwst.skymatch import skymatch
from jwst.resample import ResampleStep
from jwst.pipeline import Image2Pipeline, Image3Pipeline
from jwst.skymatch.skyimage import SkyImage
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.associations import asn_from_list
from jwst import datamodels
from jwst.datamodels import ImageModel

# Box download imports 
from astropy.utils.data import download_file
from pathlib import Path
from shutil import move
from os.path import splitext

from ci_watson.artifactory_helpers import get_bigdata

### Print pipeline version number

In [4]:
jwst.__version__ 

'1.7.2'

### Add helper script for plotting scatter plot of magnitudes

In [5]:
def create_scatterplot(catalog_colx, catalog_coly, title=None):
    ''' Function to generate a generic scatterplot.
    '''
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot()
    ax.scatter(catalog_colx,catalog_coly) 
    plt.xlabel(catalog_colx.name)
    plt.ylabel(catalog_coly.name)

    
    if title:
        plt.title(title)

<a id="data_load"></a>
# Loading the Data

In [6]:
# Read in new dataset from Box

def get_box_files(file_list):
    for box_url,file_name in file_list:
        if 'https' not in box_url:
            box_url = 'https://stsci.box.com/shared/static/' + box_url
        downloaded_file = download_file(box_url, timeout=600)
        if Path(file_name).suffix == '':
            ext = splitext(box_url)[1]
            file_name += ext
        move(downloaded_file, file_name)

# Use F560W data set
        
file_urls = ['https://stsci.box.com/shared/static/g8ozd2wkdo9klrtkafpe9qegpzhpd66a.fits', 
             'https://stsci.box.com/shared/static/ym75nbqdi32t9wnbanm6s57itqixniyc.fits', 
             'https://stsci.box.com/shared/static/0bqh6ijsc6kd7cyv6gtnj9kvly9t885f.fits',
             'https://stsci.box.com/shared/static/tywadpqcjhp9yxyv0sy91hc79red92xm.fits']


filenames = ['det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits', 
             'det_image_seq2_MIRIMAGE_F560Wexp1_rate.fits', 
             'det_image_seq3_MIRIMAGE_F560Wexp1_rate.fits',
             'det_image_seq4_MIRIMAGE_F560Wexp1_rate.fits']


# Test with F770W data set

#file_urls = ['https://stsci.box.com/shared/static/8573htf1p8mhk4e49z9b483dp975y3lz.fits',
#             'https://stsci.box.com/shared/static/p69h7uzlmqwlzf6kqqbf4lu9ibz28tc6.fits',
#             'https://stsci.box.com/shared/static/js8k6j20rek1oago057wxn08dhuticij.fits',
#             'https://stsci.box.com/shared/static/es253mc3m1sptusj9c9ctl1blidr1qdl.fits',
#             'https://stsci.box.com/shared/static/lpcgp7jkq6lmjj2lxpk63jz9u05qgk3a.cat',
#             'https://stsci.box.com/shared/static/15j6hkjc28zow4rjepob2t75dl887zs3.cat']


#file_names = ['det_image_seq1_MIRIMAGE_F770Wexp1_rate.fits', 
#             'det_image_seq2_MIRIMAGE_F770Wexp1_rate.fits', 
#             'det_image_seq3_MIRIMAGE_F770Wexp1_rate.fits',
#             'det_image_seq4_MIRIMAGE_F770Wexp1_rate.fits',
#             'input_sim_stars.cat',
#             'input_sim_galaxies.cat']

box_download_list = [(url,name) for url,name in zip(file_urls,filenames)]


get_box_files(box_download_list)

### Run rate files through calwebb_image2 to obtain calibrated images for further testing

In [7]:
# Run Calwebb_image2 on output files from detector1
ratefiles = glob('*rate.fits')    
    
print('There are ', len(ratefiles), ' images.')
    
callist = []

# cycle through files
for im in ratefiles:
    pipe2 = Image2Pipeline()
    
    rampfile = ImageModel(im)
    filename = rampfile.meta.filename
    # Set pipeline parameters

    pipe2.save_results = True
    pipe2.output_file = filename +'_cal.fits'
    pipe2.resample.save_results = True
    pipe2.suffix = None

    calfile = pipe2.run(rampfile)

    callist.append(calfile)

print(callist)

2022-09-20 13:30:11,649 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.


2022-09-20 13:30:11,650 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-09-20 13:30:11,651 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-09-20 13:30:11,653 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.


2022-09-20 13:30:11,654 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.


...

2022-09-20 13:30:12,643 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmp4ujd505z/det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits_cal.fits', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_bsub': False, 'steps': {'bkg_subtract': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_combined_background': False, 'sigma': 3.0, 'maxiters': None, 'wfss_mmag_extract': None}, 'assign_wcs': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits'

2022-09-20 13:30:12,651 - stpipe.Image2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits' reftypes = ['area', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'photom', 'regions', 'sflat', 'specwcs', 'wavelengthrange', 'wfssbkg']


2022-09-20 13:30:12,720 - CRDS - INFO -  Fetching  /tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap   26.4 K bytes  (1 / 13 files) (0 / 124.3 K bytes)


Traceback (most recent call last):
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 347, in local_bestrefs
    return hv_best_references(context, parameters, reftypes)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 367, in hv_best_references
    ctx = get_symbolic_mapping(context_file, cached=True)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 670, in get_symbolic_mapping
    return get_pickled_mapping(   # reviewed
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks

CrdsDownloadError: Failed caching mapping files: Error fetching data for 'jwst_nirspec_superbias_0051.rmap' at CRDS server 'https://jwst-crds.stsci.edu' with mode 'http' : [Errno 28] No space left on device: '/tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap'

In [ ]:
#input_files=[]
#input_files=[input_file1,input_file2,input_file3,input_file4]

imlist1=['det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq2_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq3_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq4_MIRIMAGE_F560Wexp1_cal.fits']

In [ ]:
# Look at one image
im_file = ImageModel('det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits')

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=5)
plt.colorbar()

### Set parameters for tests
Set parameters to use both in pipeline runs and for regions to examine for background area stats.

For MIRI, the FWHM values are dependent on filter and should be set using the table below:

|Filter | FWHM  |
|-------| -------|
|F560W      |     1.636		|	
|F770W      |     2.187		|
|F1000W     |     2.888		|
|F1130W     |     3.318		|
|F1280W     |     3.713		|
|F1500W     |     4.354		|
|F1800W     |     5.224		|
|F2100W     |     5.989		|
|F2550W     |     7.312		|
|F2550WR    |     7.312	    | 


In [ ]:
# Choose region of image to examine for stats (preferably no bright sources)
xmin = 700
xmax = 750
ymin = 400
ymax = 450

# parameters for calwebb_image3 runs
snr = 5
fwhm = 1.7

<a id="associations_ID"></a>
### Set up association files
The level three pipeline relies on an association file to specify which files are to be combined and provide the output file name.

[Top of Page](#title_ID)

In [ ]:
# use asn_from_list to create association table
cal_list=imlist1
asn = asn_from_list.asn_from_list(cal_list, rule=DMS_Level3_Base, product_name='skymatch_combined.fits')

# use this if you need to add non'science' exposure types
#asn['products'][0]['members'][1]['exptype'] = 'background'
#asn['products'][0]['members'][2]['exptype'] = 'sourcecat'

# dump association table to a .json file for use in image3
with open('skymatch_asnfile.json', 'w') as fp:
    fp.write(asn.dump()[1])

skymatch_json_file='skymatch_asnfile.json'
    
json_file = skymatch_json_file
file_list = []
file_list2 = []
with open(json_file) as json_data:
    d = json.load(json_data)
    members = d['products'][0]['members']
    for item in np.arange(0,len(members)):
        file_list.append(members[item]['expname'])
        file_list2.append(members[item]['expname'][:-5]+"_skymatch.fits")

    
asn2 = asn_from_list.asn_from_list(file_list2, rule=DMS_Level3_Base, product_name=d['products'][0]['name'])

# use this if you need to add non'science' exposure types
#asn['products'][0]['members'][1]['exptype'] = 'background'
#asn['products'][0]['members'][2]['exptype'] = 'sourcecat'

# dump association table to a .json file for use in image3
with open('skymatch_asnfile2.json', 'w') as fp:
    fp.write(asn2.dump()[1])
    
skymatch_json_file2='skymatch_asnfile2.json'

### Modify average background level of input images

In [ ]:
infile01_1 = imlist1[0]
infile01_2 = imlist1[1]
infile02_1 = imlist1[2]
infile02_2 = imlist1[3]

In [ ]:
img01_1 = datamodels.open(infile01_1)
img01_2 = datamodels.open(infile01_2)
img02_1 = datamodels.open(infile02_1)
img02_2 = datamodels.open(infile02_2)

In [ ]:
data01_1 = img01_1.data
data01_2 = img01_2.data
data02_1 = img02_1.data
data02_2 = img02_2.data

#data01_1[data01_1<=0.3]=np.nan
#data01_2[data01_2<=0.3]=np.nan
#data02_1[data02_1<=0.3]=np.nan
#data02_2[data02_2<=0.3]=np.nan

data01_1_orig = np.copy(img01_1.data)
data01_2_orig = np.copy(img01_2.data)
data02_1_orig = np.copy(img02_1.data)
data02_2_orig = np.copy(img02_2.data)

In [ ]:
# check mean values of background 
print('Mean:', np.nanmean(data01_1_orig[data01_1_orig<=4.0]),' Standard deviation: ',np.std(data01_1_orig[data01_1_orig<=4.0]))
print('Mean:', np.nanmean(data01_2_orig[data01_2_orig<=4.0]),' Standard deviation: ',np.std(data01_2_orig[data01_2_orig<=4.0]))
print('Mean:', np.nanmean(data02_1_orig[data02_1_orig<=4.0]),' Standard deviation: ',np.std(data02_1_orig[data02_1_orig<=4.0]))
print('Mean:', np.nanmean(data02_2_orig[data02_2_orig<=4.0]),' Standard deviation: ',np.std(data02_2_orig[data02_2_orig<=4.0]))

In [ ]:
# creating a background image with specified mean and gaussian noise with sigma = 1.0
bkg_img_noise_neg2 = np.random.normal(-2,2*0.2,data01_1.shape)
bkg_img_noise_2 = np.random.normal(2,2*0.2,data01_1.shape)
bkg_img_noise_3 = np.random.normal(3,3*0.2,data01_1.shape)
bkg_img_noise_4 = np.random.normal(4,4*0.2,data01_1.shape)
bkg_img_noise_5 = np.random.normal(5,5*0.2,data01_1.shape)
bkg_img_noise_7 = np.random.normal(7,7*0.2,data01_1.shape)

Add offset noise to the background of the images for easier background visualization

In [ ]:
# adding the new background with specified mean and gaussian noise (above) to image 
img01_1.data=bkg_img_noise_2 + img01_1.data
img01_2.data=bkg_img_noise_3 + img01_2.data
img02_1.data=bkg_img_noise_5 + img02_1.data
img02_2.data=bkg_img_noise_2 + img02_2.data

In [ ]:
# checking the mean and standard deviations of the new background values
print('Mean:',np.nanmean(img01_1.data),' Standard deviation: ',np.nanstd(img01_1.data))
print('Mean:',np.nanmean(img01_2.data),' Standard deviation: ',np.nanstd(img01_2.data))
print('Mean:',np.nanmean(img02_1.data),' Standard deviation: ',np.nanstd(img02_1.data))
print('Mean:',np.nanmean(img02_2.data),' Standard deviation: ',np.nanstd(img02_2.data))

In [ ]:
img01_1.save(file_list2[0],overwrite=True)
img01_2.save(file_list2[1],overwrite=True)
img02_1.save(file_list2[2],overwrite=True)
img02_2.save(file_list2[3],overwrite=True)

In [ ]:
# Look at one image
im_file = ImageModel(file_list2[0])

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

<a id="pipeline_ID"></a>
## Run Pipeline steps in calwebb_image3

Run the pipeline on the data for different sets of parameters

## 1. Run skymatch step on images with skymatch method = 'local'

Notes: ‘local’: compute sky background values of each input image or group of images (members of the same “exposure”). A single sky value is computed for each group of images. This method simply computes the mean/median/mode/etc. value of the “sky” separately in each input image. This will resulted in subtracted background being near zero in the combined image, as each image has it's individual background subtracted.

match_down specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True)

'Subtract' specifies whether the computed sky background values are to be subtracted from the images in the skymatch step. (Default = False) Currently, if not done in skymatch, the subtraction is performed in resample. 

### 1.1 Run skymatch with sky match method = local. Subtract = False, match_down = True.


In [ ]:
# skymatch, local, subtract= False, all 4 images used
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='local', subtract=False,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skipping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'local' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = False # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True

pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
# Check values out of skymatch step. With subtract = False, values should be equal.

img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('Original and new levels should be equal. If subtract=False, skymatch should not subtract.')
print('Mean: original, new (local)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('The resample step should subtract the individual backgrounds found in the skymatch step.')
print('Since the local individual background level is subtracted from each image, final expected result is near zero.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

#### Look at photometry out of source catalog to compare across tests.


In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
cat_local = catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')


### 1.2  Skymatch method = 'Local', subtract= True. 

Skymatch step should subtract the background. Resample background level should match the skymatch background level since subtraction has already been done

In [ ]:
# skymatch, local, subtract = True, all 4 images used
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='local', subtract=True,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'local' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
#match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = True # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
#pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('With subtract=True, the new value should show that the background value was subtracted.')
print('Mean: original, new (local)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('The background value was subtracted in the skymatch step, no additional subtraction should be done here.')
print('This value should match the values in the new column above.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')


In [ ]:
#Look at the sources found on the image
viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
plt.title('Sources found with Local method')
plt.imshow(viz2(im_file.data), origin='lower')
plt.colorbar()
plt.scatter(catalog['xcentroid'], catalog['ycentroid'],lw=1, s=10,color='red')

## 2. Run skymatch for skymethod = 'global'.


##### Notes: ‘global’: compute a common sky value for all input image and groups of images. In this setting match will compute sky values for each input image/group, find the minimum sky value, and then it will set (and/or subtract) sky value of each input image to this minimum value. This method may be useful when input images have been already matched. The 'global' method computes the minimum sky value across all input images and/or groups. That sky value is then considered to be the background in all input images.

This calculates an average sky across all images and subtracts that average from all images. There should be a warning for users for this step that makes it clear that this should not be used on images with differences in background levels.

### 2.1 skymethod = 'global' with subtraction = False

In [ ]:
# skymatch, local
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='local', subtract=False,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'global' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
#match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = False # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
#pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('There should be no subtraction at this step, with subtraction = False.')
print('Mean: original, new (local)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('Global finds a single value for the background, and subtracts that value from all images.')
print('The value listed here should be the average of the subtracted sky values across images.')
print('The combined image will look messy as the same value was subtracted from the image with background = 5')
print('and from the images with background = 2. This shows the differing backgrounds strongly.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
cat_global = catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')


In [ ]:
#Look at the sources found on the image
viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
plt.title('Sources found with Global method')
plt.imshow(viz2(im_file.data), origin='lower')
plt.colorbar()
plt.scatter(catalog['xcentroid'], catalog['ycentroid'],lw=1, s=10,color='red')

### 2.2 skymethod = 'global' with subtraction = True

In [ ]:
# skymatch, global, subtract = True
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='global', subtract=True,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'global' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
#match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = True # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
#pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('Global finds a single value for the background, and subtracts that value from all images.')
print('The values listed here should be the average of the subtracted sky values across images subtracted from each image.')
print('Mean: original, new (global)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('Global finds a single value for the background, and subtracts that value from all images.')
print('The value listed here should be the average of the subtracted sky values across images.')
print('The combined image will look messy as the same value was subtracted from the image with background = 5')
print('and from the images with background = 2. This shows the differing backgrounds strongly.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')


## 3. Run tests on Skymatch method 'Match'
##### Notes: ‘match’: compute differences in sky values between images and/or groups in (pair-wise) common sky regions. In this case computed sky values will be relative (delta) to the sky computed in one of the input images whose sky value will be set to (reported to be) 0. This case matches to the lowest sky value.

Based on whether match_down is set to True or False, Match will calculate the difference between the lowest background level or highest level (respectively), and subtract the difference between the calculated level and the 'matched' level.

This is the preferred default option as it subtracts the differences between the background levels, normalizing all of the backgrounds to a common level (either lowest or highest background), rather than subtracting off all of the background.

### 3.1 skymethod = 'match', match_down=True, subtract=True

In [ ]:
# skymatch, match down, subtract = True
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='match', match_down=True,subtract=True,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'match' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = True # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('There should be no subtraction at this step, with subtraction = False.')
print('Mean: original, new (match down)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('For the method match with match_down= True, the value of the background should match the minimum of the individual backgrounds.')
print('This value should match the values in the new column above. Subtraction was done in skymatch step.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')


### ** Check that all output (new) sky values are equal to the lowest original sky value 

### 3.2 skymethod = 'match', match_down=False, subtract = True

In [ ]:
# skymatch, match up, subtract = True
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='match', match_down=False,subtract=True,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'match' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
match_down = False # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = True # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('With match_down = False, match will match the backgrounds of all to the highest background.')
print('With subtract = True, the background matching is done at the skymatch step.')
print('Mean: original, new (match up)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('For the method match with match_down=False, the value of the background should match the maximum of the individual backgrounds')
print('This value should match the values in the new column above. Subtraction was done in skymatch step.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=7)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')

### ** Check that all output (new) sky values are equal to the highest original sky value 

### 3.3 skymethod = 'match', match_down = True, subtract = False

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'match' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = False # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('With subtract=False, no subtraction should be done here.')
print('Mean: original, new (match down)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('With the match method, subtract=False and match_down=True, the background value should match to the lowest individual background.')
print('This value should match the minimum values in the new column above.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
cat_match = catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')

In [ ]:
#Look at the sources found on the image
#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
plt.title('Sources found with Match method')
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='gray', vmin=0, vmax=4)
plt.colorbar()
plt.scatter(catalog['xcentroid'], catalog['ycentroid'],lw=1, s=10,color='red')

## 4. Test skymatch method 'global+match'
##### Notes: ‘global+match’: first use ‘match’ method to equalize sky values between images and then find a minimum “global” sky value in all input images.

The behavior of this step is that it subtracts all background. More documentation is needed on how exactly it gets to that point.

### 4.1 skymethod = 'global+match', subtract = True

In [ ]:
# skymatch, global+match, subtract = True
#jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='global+match', match_down=True,subtract=True,output_file='MIRI',save_results=True)

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'global+match' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
#match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = True # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
#pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('Mean: original, new (global+match)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

#print('Did resample subtract a background level, and what was it?')
print('This value should match the values in the new column above. Subtraction was done in skymatch step.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')

### 4.2 skymatch = global+match, subtract = False

In [ ]:
# Test running just skymatch and resample and skiping other steps.

# Run Calwebb_image3 on the association table
    
# set any specific parameters
skymethod = 'global+match' # sky computation algorithm to be used. Allowed values: {local, global, match, global+match} (Default = global+match
#match_down = True # Specifies whether the sky differences should be subtracted from images with higher sky values (match_down = True) in order to match the image with the lowest sky or sky differences should be added to the images with lower sky values to match the sky of the image with the highest sky value (match_down = False). (Default = True
subtract = False # Specifies whether the computed sky background values are to be subtracted from the images. (Default = False   
    
pipe3=Image3Pipeline()    
pipe3.tweakreg.skip = True
pipe3.outlier_detection.skip = True

pipe3.skymatch.skymethod = skymethod
#pipe3.skymatch.match_down = match_down
pipe3.skymatch.subtract = subtract
pipe3.skymatch.output_file = 'MIRI'
pipe3.skymatch.save_results = True

pipe3.resample.save_results = True
pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm

pipe3.save_results = True

# run Image3

image = pipe3.run(skymatch_json_file2)
print('Image 3 pipeline finished.')

In [ ]:
img01_1_new = datamodels.open('MIRI_0_skymatch.fits')
img01_2_new = datamodels.open('MIRI_1_skymatch.fits')
img02_1_new = datamodels.open('MIRI_2_skymatch.fits')
img02_2_new = datamodels.open('MIRI_3_skymatch.fits')

print('Mean: original, new (global+match)')
print('Mean:',np.nanmean(img01_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img01_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img01_2_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_1.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_1_new.data[ymin:ymax,xmin:xmax]))
print('Mean:',np.nanmean(img02_2.data[ymin:ymax,xmin:xmax]),',', np.nanmean(img02_2_new.data[ymin:ymax,xmin:xmax]))

### ** Check that all output (new) sky values are around zero (within a standard deviation)

In [ ]:
# Look at combined image
im_file = ImageModel('skymatch_combined_i2d.fits')

print('Did resample subtract a background level, and what was it?')
#print('This value should match the values in the new column above. Subtraction was done in skymatch step.')
print('Mean:',np.nanmean(im_file.data[ymin:ymax,xmin:xmax]))

# read in ecsv photom file
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_file.data), origin='lower')
plt.imshow(im_file.data, origin='lower', cmap='rainbow', vmin=0, vmax=4)
plt.colorbar()

In [ ]:
# Get name of output catalog
catfile = 'skymatch_combined_cat.ecsv'
print(catfile)

# Read in catalog
#catalog = Table.read(catfile)
catalog = table.Table.read(catfile, format='ascii', comment='#')
cat_global_match = catalog

# Plot AB mag values
create_scatterplot(catalog['label'], catalog['aper_total_abmag'],title='Total AB mag')

In [ ]:
#Look at the sources found on the image
viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
plt.title('Sources found with Global+Match method')
plt.imshow(viz2(im_file.data), origin='lower')
plt.colorbar()
plt.scatter(catalog['xcentroid'], catalog['ycentroid'],lw=1, s=10,color='red')

### Compare photometry across the different skymatch methods

Determine whether the different methods give different photometric results. Ideally, the methods won't affect the output photometry, and the symbols should fall on top of each other for the sources plotted.


In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.subplot()

plt.title('Photometry across methods')
plt.xlabel('x position')
plt.ylabel('aper_total_abmag')
ax.scatter(cat_local['xcentroid'],cat_local['aper_total_abmag'], c='black', marker='o', label='Local')
ax.scatter(cat_global['xcentroid'], cat_global['aper_total_abmag'], c='green', marker='v', label='Global')
ax.scatter(cat_match['xcentroid'], cat_match['aper_total_abmag'], c='blue', marker='x', label='Match')
ax.scatter(cat_global_match['xcentroid'], cat_global_match['aper_total_abmag'], c='magenta', marker='d', label='Global+Match')
ax.legend()

<a id="about_ID"></a>
## About this Notebook
**Author:** T. Temim and M. Cracraft, MIRI team
<br>**Updated On:** 01/21/2022